In [ ]:
#comparison python wilcoxon and r wilcoxon 

py_result_wilcx = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_wilcoxon_test.csv")
r_result_wilcx = pd.read_csv("LR2TF_test_run\\results\\TF_results\\control\\all_specificmarker__control.csv")

py_result_wilcx = py_result_wilcx[py_result_wilcx["pvals_adj"] < 0.05]
r_result_wilcx = r_result_wilcx[r_result_wilcx["p_val_adj"] < 0.05]

py_result_wilcx = py_result_wilcx.rename(columns={"names": "gene", "group": "cluster"})

df_output_py = pd.DataFrame()

for i in range(len(py_result_wilcx)):
    a = py_result_wilcx["gene"].iloc[i]
    for j in range(len(r_result_wilcx)):
        b = r_result_wilcx["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result_wilcx.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result_wilcx)):
    a = r_result_wilcx["gene"].iloc[i]
    for j in range(len(py_result_wilcx)):
        b = py_result_wilcx["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result_wilcx.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_wilcoxon_r_same_genes_control_filtered.csv")

#comparison python t test and r wilcoxon

py_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_t_test_overestim_test.csv")
r_result = pd.read_csv("LR2TF_test_run\\results\\TF_results\\control\\all_specificmarker__control.csv")

py_result = py_result[py_result["pvals_adj"] < 0.05]
r_result = r_result[r_result["p_val_adj"] < 0.05]
py_result = py_result.rename(columns={"names": "gene", "group": "cluster"})


df_output_py = pd.DataFrame()

for i in range(len(py_result)):
    a = py_result["gene"].iloc[i]
    for j in range(len(r_result)):
        b = r_result["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result)):
    a = r_result["gene"].iloc[i]
    for j in range(len(py_result)):
        b = py_result["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result.iloc[i]

df_output_r = df_output_r.T

#df_output = pd.concat([df_output_py, df_output_r], keys= ["gene"])
df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
#df_output = py_result_wilcx.rename(columns={"gene": "gene_py", "Unnamed: 0": "gene_R"})

df_output.to_csv("tf_comparison_r_py/py_t_test_r_same_genes_control_filtered.csv")

# py wilcox vs py t test
py_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_t_test_overestim_test.csv")
r_result = pd.read_csv("script_test\\TF_results\\control\\control_specific_markers_wilcoxon_test.csv")


py_result = py_result[py_result["pvals_adj"] < 0.05]
r_result = r_result[r_result["p_val_adj"] < 0.05]
py_result = py_result.rename(columns={"names": "gene", "group": "cluster"})


df_output_py = pd.DataFrame()

for i in range(len(py_result)):
    a = py_result["gene"].iloc[i]
    for j in range(len(r_result)):
        b = r_result["gene"].iloc[j]
        if a == b:
            df_output_py[i] = py_result.iloc[i]
            

df_output_py = df_output_py.T

df_output_r = pd.DataFrame()
for i in range(len(r_result)):
    a = r_result["gene"].iloc[i]
    for j in range(len(py_result)):
        b = py_result["gene"].iloc[j]
        if a == b:
            df_output_r[i] = r_result.iloc[i]

df_output_r = df_output_r.T

df_output = pd.merge(df_output_py, df_output_r, on="gene", suffixes=("_py", "_r"))
df_output.to_csv("tf_comparison_r_py/py_t_test_py_wilcox_same_genes_control_filtered.csv")

In [ ]:

    
    #  df = add_entry_to_CTR_dataframe(tf_activities.iloc[row, 2],
    #                                         tf_activities.iloc[row, 2],
    #                                         tf_activities.iloc[row, 0],
    #                                         ligand,
    #                                         'Transcription Factor',
    #                                         'Ligand',
     #                                        tf_activities.iloc[row, 3])
      
 #     

In [ ]:
gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
obs = sub_object[:,gene_ids].X.toarray()
obs = np.expm1(obs)
sub_object = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs["new_annotation"])
sub_object = sub_object.groupby(level=0, observed=False).mean()
#sub_object.T.to_csv(outpath + name + "average_gene_expression_by_cluster_exp.csv")

sub_object.T


In [ ]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
      #  anndataobject.rename_categories("protocol", ["PMFMF2", "control"])

tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs

tf_activities_sub = tf_activities[anndataobject.obs.protocol == "PMF,MF2"]
print(tf_activities_sub)

In [ ]:
tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs
tf_activities = tf_activities[tf_activities.obs.protocol == "control"]
tf_activities.obs

In [ ]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")


tf_activities = ad.read_csv("decoupler_results.csv")
anndataobject.obsm['ulm_estimate'] = tf_activities
print(anndataobject.obsm["ulm_estimate"].X)
for name in anndataobject.obs.protocol.unique():
##    subobject = anndataobject[anndataobject.obs.protocol == name]
#print(type(subobject)) 
#print(type(anndataobject))  
#print(subobject.X)

